In [ ]:
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
import json
import os

Connect to Kafka

In [ ]:
producer = KafkaProducer(bootstrap_servers=['100.25.219.224:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

Load data

In [ ]:
df = pd.read_csv("indexProcessed.csv")

Start streaming

In [ ]:
while True:
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)
    sleep(1)

Flush data when done

In [ ]:
producer.flush() #clear data from kafka server